In [71]:
# Necessary imports
import os
import numpy as np
import tensorflow as tf
import keras
import nengo_dl
import random
from tensorflow.python.keras import Input, Model
import nengo
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, Conv3D
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras import backend as K

In [72]:
dataset_path = os.path.join('dataset_result')
files = [os.path.join(dataset_path, 'P{:02d}.npz'.format(i+1))
         for i in range(18)] # P01 - P18 files

In [73]:
# set seed to produce consistent result
seed = 2
np.random.seed(seed)
tf.random.set_seed(seed)

In [74]:
def cnn_model_2():
    inp = Input(shape=(14, 36, 10), name='input_layer')
    conv1 = Conv2D(filters=32, kernel_size=(5, 5), activation=tf.nn.relu, padding='same')(inp)
    dropout1 = Dropout(0.2, seed=seed)(conv1)
    avg_pool1 = AveragePooling2D(pool_size=(2, 2))(dropout1)
    conv2 = Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu)(avg_pool1)
    dropout2 = Dropout(0.2, seed=seed)(conv2)
    avg_pool2 = AveragePooling2D(pool_size=(2, 2))(dropout2)
    flatten = Flatten()(avg_pool2)
    dense1 = Dense(512, activation=tf.nn.relu)(flatten)
    dropout3 = Dropout(0.2, seed=seed)(dense1)
    dense2 = Dense(256, activation=tf.nn.relu)(dropout3)
    output = Dense(2, activation=tf.nn.softmax, name='output_layer')(dense2)

    return Model(inputs=inp, outputs=output)

In [75]:
def run_tensorflow_only(model, x_train, y_train, x_test, y_test, epochs=30):
    model.compile(
        optimizer='adam',
        loss=tf.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )

    model.fit(x_train, y_train, epochs=epochs)
    eval = model.evaluate(x_test, y_test)
    print('Accuracy: ', eval[1] * 100)
    return eval[1]

In [76]:
def run_ann(model, x_train, y_train, x_test, y_test, params_save_path, iteration, epochs=30):
    converter = nengo_dl.Converter(model)

    with nengo_dl.Simulator(converter.net) as simulator:
        simulator.compile(
            optimizer=keras.optimizers.Adam(),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=['accuracy'],

        )

        input_layer = converter.inputs[model.get_layer('input_layer')]
        output_layer = converter.outputs[model.get_layer('output_layer')]

        # early stop to avoid overfitting
        hist = simulator.fit(
            x={ input_layer: x_train }, y={ output_layer: y_train },
            epochs=epochs,
            shuffle=True,
            callbacks=[EarlyStopping(patience=8, verbose=1, restore_best_weights=True, monitor='probe_loss')]
        )
        print(hist.history)

        simulator.save_params(params_save_path)

        # Test trained model and print result
        eval = simulator.evaluate(x=x_test, y=y_test)
        print('{}. ANN accuracy: {:5f}%'.format(
            iteration, eval['probe_accuracy'] * 100
        ))

        return eval['probe_accuracy'] # return accuracy

In [77]:
def run_snn(model, x_test, y_test, params_load_path, iteration, timesteps=50, scale_firing_rates=1000, synapse=0.01):
    converter = nengo_dl.Converter(
        model,
        swap_activations={ tf.nn.relu: nengo.SpikingRectifiedLinear() },
        scale_firing_rates=scale_firing_rates,
        synapse=synapse
    ) # create a Nengo converter object and swap all relu activations with spiking relu

    with converter.net:
        nengo_dl.configure_settings(stateful=False)

    input_layer = converter.inputs[model.get_layer('input_layer')] # input layer for simulator
    output_layer = converter.outputs[model.get_layer('output_layer')] # output layer for simulator

    x_test_tiled = np.tile(x_test, (1, timesteps, 1)) # tile test data to timesteps

    with nengo_dl.Simulator(converter.net) as simulator:
        simulator.load_params(params_load_path)

        predictions = simulator.predict({ input_layer: x_test_tiled })[output_layer] # get results from prediction
        predictions = predictions[:,-1,:] # get last time step

        predictions = np.argmax(predictions, axis=-1) # get argmax
        y_test = np.squeeze(y_test, axis=1) # remove time dimension from labels since its not relevant
        y_test = np.argmax(y_test, axis=-1) # get argmax of y test as well for comparison

        snn_acc = (predictions == y_test).mean()

    print('{}. SNN accuracy: {:5f}%'.format(
            iteration, snn_acc * 100
        ))

    return snn_acc

In [78]:
def transform_dataset(features, labels):
    labels = labels.reshape((-1, 1))
    labels = OneHotEncoder().fit_transform(labels).toarray()
    labels = labels.reshape((labels.shape[0], 1, -1)) # flatten and add time dimension (necessary for nengo)
    features = features.reshape((features.shape[0], 1, -1)) # flatten and add time dimension

    return features, labels

In [79]:
def run_individual(file, particip_num, model, params_save_path, test_size=0.25, epochs=30, scale_firing_rates=1000,
                   synapse=0.01, timesteps=50):
    print('Running ANN and SNN for file:', file)
    dataset = np.load(file) # load numpy file containing the preprocessed data for specific participant
    features, labels = dataset['features'], dataset['labels'] # get features and labels from the numpy file

    # transform numpy arrays to be usable with nengo
    features, labels = transform_dataset(features, labels)
    x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=seed,
                                                        shuffle=True)
    print('X (train) shape:', x_train.shape, 'Y (train) shape:', y_train.shape)
    print('X (test) shape:', x_test.shape, 'Y (test) shape:', y_test.shape)

    # tf_only_acc = run_tensorflow_only(model, x_train, y_train, x_test, y_test)

    ann_acc = run_ann(model, x_train, y_train, x_test, y_test, params_save_path, particip_num,
                      epochs=epochs)

    snn_acc = run_snn(model, x_test, y_test, params_save_path, particip_num,
                      timesteps=timesteps, synapse=synapse, scale_firing_rates=scale_firing_rates)

    return ann_acc, snn_acc

In [80]:
results = {}

i = 1
params_save_dir = 'nengo_individuals_params'
os.makedirs(params_save_dir, exist_ok=True)
for file in files:
    file_name = 'P{:02d}'.format(i)
    model = cnn_model_2()
    params_save_path = os.path.join(params_save_dir, file_name)
    ann_acc, snn_acc = run_individual(file, i, model, params_save_path, epochs=30)

    results[file_name] = { 'ann_acc': ann_acc, 'snn_acc': snn_acc }
    i += 1

    K.clear_session()

Running ANN and SNN for file: dataset_result\P01.npz
X (train) shape: (90, 1, 5040) Y (train) shape: (90, 1, 2)
X (test) shape: (30, 1, 5040) Y (test) shape: (30, 1, 2)
Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/30
90/90 [==============================] - 4s 9ms/step - loss: 1.0809 - probe_loss: 1.0809 - probe_accuracy: 0.5860

c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Layer type Dropout does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:588: UserWarning: Activation type <function softmax_v2 at 0x00000187AB8F15E0> does not have a native Nengo equivalent; falling back to a TensorNode
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\simulator.py:1765: UserWarning: Number of elements (1) in ['ndarray'] does not match number of Nodes (4); consider using an explicit input dictionary in this case, so that the assignment of data to objects is unambiguous.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Layer type Dropout does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:588: User

In [81]:
for result in results.keys():
    print('{}: {}'.format(result, results[result]))

P01: {'ann_acc': 0.5333333611488342, 'snn_acc': 0.5333333333333333}
P02: {'ann_acc': 0.4761904776096344, 'snn_acc': 0.5}
P03: {'ann_acc': 0.4523809552192688, 'snn_acc': 0.47619047619047616}
P04: {'ann_acc': 0.4761904776096344, 'snn_acc': 0.47619047619047616}
P05: {'ann_acc': 0.4761904776096344, 'snn_acc': 0.5}
P06: {'ann_acc': 0.5476190447807312, 'snn_acc': 0.5476190476190477}
P07: {'ann_acc': 0.5, 'snn_acc': 0.5}
P08: {'ann_acc': 0.4761904776096344, 'snn_acc': 0.4523809523809524}
P09: {'ann_acc': 0.523809552192688, 'snn_acc': 0.5714285714285714}
P10: {'ann_acc': 0.5, 'snn_acc': 0.5}
P11: {'ann_acc': 0.523809552192688, 'snn_acc': 0.5238095238095238}
P12: {'ann_acc': 0.523809552192688, 'snn_acc': 0.5238095238095238}
P13: {'ann_acc': 0.4761904776096344, 'snn_acc': 0.47619047619047616}
P14: {'ann_acc': 0.4761904776096344, 'snn_acc': 0.47619047619047616}
P15: {'ann_acc': 0.4047619104385376, 'snn_acc': 0.42857142857142855}
P16: {'ann_acc': 0.5476190447807312, 'snn_acc': 0.5476190476190477}


In [82]:
ann = [results[file_name]['ann_acc'] for file_name in results]
snn = [results[file_name]['snn_acc'] for file_name in results]
ann_acc_avg, snn_acc_avg = np.average(ann), np.average(snn)
ann_max, snn_max = max(ann), max(snn)

print('Average accuracy ANN:', ann_acc_avg)
print('Average accuracy SNN:', snn_acc_avg)
print('Max accuracy ANN:', ann_max)
print('Max accuracy SNN:', snn_max)

Average accuracy ANN: 0.49920635753207737
Average accuracy SNN: 0.5005291005291005
Max accuracy ANN: 0.5714285969734192
Max accuracy SNN: 0.5714285714285714
